In [ ]:
from math import inf
import sys, os

HUMAN = 1
COMP = -1

board = [[0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]]

MSG = "Welcome to Unbeatable Tic Tac Toe.\n" \
      "Our A.I can foresee your moves ahead.\n" \
      "Are you sure to continue? (y/n)"


def is_within_bounds(x, y, size=5):
    return 0 <= x < size and 0 <= y < size


def check_sequence(state, length, x, y, dx, dy, scored_positions, player):
    size = len(state)
    count = 0
    blocked_ends = 0

    # Check for consecutive moves
    for step in range(length):
        nx, ny = x + step * dx, y + step * dy
        if is_within_bounds(nx, ny, size):
            if state[nx][ny] == player and not scored_positions[nx][ny]:
                count += 1
            else:
                break
        else:
            break

    if count == length:
        # Check the ends
        prev_x, prev_y = x - dx, y - dy
        next_x, next_y = x + length * dx, y + length * dy

        if not is_within_bounds(prev_x, prev_y, size) or (is_within_bounds(prev_x, prev_y, size) and state[prev_x][prev_y] != 0):
            blocked_ends += 1
        if not is_within_bounds(next_x, next_y, size) or (is_within_bounds(next_x, next_y, size) and state[next_x][next_y] != 0):
            blocked_ends += 1

        score = 0
        if blocked_ends == 2:
            score = 0
        elif blocked_ends == 1:
            if length == 2:
                score = 25
                print("Blocked ends: 1, Length: 2, Score: 25")
            elif length == 3:
                score = 100
                print("Blocked ends: 1, Length: 3, Score: 100")
            elif length == 4:
                score = 500
                print("Blocked ends: 1, Length: 4, Score: 500")
        else:
            if length == 2:
                score = 50
                print("Blocked ends: 0, Length: 2, Score: 50")
            elif length == 3:
                score = 300
                print("Blocked ends: 0, Length: 3, Score: 300")
            elif length == 4:
                score = 1000
                print("Blocked ends: 0, Length: 4, Score: 1000")

        # Mark positions as scored
        for step in range(length):
            nx, ny = x + step * dx, y + step * dy
            if is_within_bounds(nx, ny, size):
                scored_positions[nx][ny] = True

        return score

    return None


def calculate_total_score(state, player):
    size = len(state)
    total_score = 0

    directions = [
        (1, 0),
        (0, 1),
        (1, 1),
        (1, -1)
    ]

    # Array to mark cells that are part of a scored sequence
    scored_positions = [[False] * size for _ in range(size)]

    for x in range(size):
        for y in range(size):
            if state[x][y] == player and not scored_positions[x][y]:
                for dx, dy in directions:
                    # Check for length 4 first
                    score = check_sequence(state, 4, x, y, dx, dy, scored_positions, player)
                    if score is not None:
                        total_score += score
                        continue

                    # Check for length 3 next
                    score = check_sequence(state, 3, x, y, dx, dy, scored_positions, player)
                    if score is not None:
                        total_score += score
                        continue

                    # Check for length 2 last
                    score = check_sequence(state, 2, x, y, dx, dy, scored_positions, player)
                    if score is not None:
                        total_score += score

    return total_score


def evaluate(state):
    """Perform heuristic evaluation of the board."""
    if wins(state, COMP):
        score = -99999999
    elif wins(state, HUMAN):
        score = 99999999
    else:
        score = calculate_total_score(state, HUMAN) - calculate_total_score(state, COMP)
    return score


def empty_cells(state):
    """Extract the remainder of the board."""
    cells = []  # It contains all empty cells

    for i, row in enumerate(state):
        for j, col in enumerate(row):
            if state[i][j] == 0:
                cells.append([i, j])

    return cells


def wins(state, player):
    """Check all winning conditions for a player."""
    # Check rows
    for row in state:
        for i in range(len(row) - 4):
            if row[i] == row[i + 1] == row[i + 2] == row[i + 3] == row[i + 4] == player:
                return True

    # Check columns
    for j in range(len(state[0])):
        for i in range(len(state) - 4):
            if state[i][j] == state[i + 1][j] == state[i + 2][j] == state[i + 3][j] == state[i + 4][j] == player:
                return True

    # Check diagonals
    for i in range(len(state) - 4):
        for j in range(len(state[0]) - 4):
            if state[i][j] == state[i + 1][j + 1] == state[i + 2][j + 2] == state[i + 3][j + 3] == state[i + 4][j + 4] == player:
                return True
            if state[i][j + 4] == state[i + 1][j + 3] == state[i + 2][j + 2] == state[i + 3][j + 1] == state[i + 4][j] == player:
                return True

    return False


def game_over(state):
    """Check game over condition."""
    return wins(state, HUMAN) or wins(state, COMP)


def clean():
    """Clear system terminal."""
    os_name = sys.platform.lower()
    if 'win' in os_name:
        os.system('cls')
    else:
        os.system('clear')


def minimax(position, depth, alpha, beta, maximizingPlayer):
    if maximizingPlayer:
        best = [-1, -1, float('-inf')]
    else:
        best = [-1, -1, float('inf')]

    if depth == 0 or game_over(position):
        score = evaluate(position)
        return [-1, -1, score]

    if maximizingPlayer:
        for cell in empty_cells(position):
            x, y = cell
            new_position = [row[:] for row in position]  # Make a copy of the current position
            new_position[x][y] = COMP  # COMP is the maximizing player
            eval = minimax(new_position, depth - 1, alpha, beta, False)
            if eval[2] > best[2]:
                best = [x, y, eval[2]]
            alpha = max(alpha, eval[2])
            if beta <= alpha:
                break
        return best
    else:
        for cell in empty_cells(position):
            x, y = cell
            new_position = [row[:] for row in position]  # Make a copy of the current position
            new_position[x][y] = HUMAN  # HUMAN is the minimizing player
            eval = minimax(new_position, depth - 1, alpha, beta, True)
            if eval[2] < best[2]:
                best = [x, y, eval[2]]
            beta = min(beta, eval[2])
            if beta <= alpha:
                break
        return best


def human_turn(state):
    moves = {}
    BOARD_SIZE = 5
    for i in range(BOARD_SIZE):
        for j in range(BOARD_SIZE):
            moves[i * BOARD_SIZE + j + 1] = [i, j]

    remain = empty_cells(state)
    isTurn = True
    print("Human Turn")
    while isTurn:
        try:
            move = int(input("Enter your move (1-25): "))
            # When the player move is valid
            if moves.get(move) in remain:
                x, y = moves.get(move)
                state[x][y] = HUMAN
                isTurn = False
            else:  # Otherwise
                print("Bad Move, try again.")
        except ValueError:
            print("Blank space and string are prohibited, please enter (1-25)")

    # While-else loop, this code below will run after successful loop.
    else:
        clean()
        print(render(state))


def ai_turn(state):
    depth = 5
    alpha = -float('inf')
    beta = float('inf')
    remain = empty_cells(state)
    row, col, score = minimax(state, depth, alpha, beta, COMP)  # The optimal move for computer
    if [row, col] in remain:
        state[row][col] = COMP
    print("A.I Turn")
    print(render(state))  # Show result board


def render(state):
    """Render the board state to stdout."""
    legend = {0: " ", 1: "X", -1: "O"}
    state = list(map(lambda x: [legend[y] for y in x], state))
    result = "{}\n{}\n{}\n{}\n{}\n".format(*state)
    return result


def main():
    """Main function: Function that will run at start."""
    print(MSG)

    start = False
    while not start:
        confirm = input("")

        if confirm.lower() in ["y", "yes"]:
            start = True
        elif confirm.lower() in ["n", "no"]:
            sys.exit()
        else:
            print("Please enter 'y' or 'n'")

    else:
        clean()
        print("Game is settled!\n")
        print(render(board), end="\n")

    while not wins(board, COMP) and not wins(board, HUMAN):
        human_turn(board)
        if len(empty_cells(board)) == 0: break
        ai_turn(board)

    if wins(board, COMP):
        print("A.I wins, 'I see through your moves'")
    elif wins(board, HUMAN):
        print("Human wins, 'How can I lose to a human?'")
    else:
        print("It's a Draw. No one wins")


if __name__ == '__main__':
    main()

Welcome to Unbeatable Tic Tac Toe.
Our A.I can foresee your moves ahead.
Are you sure to continue? (y/n)
y
Game is settled!

[' ', ' ', ' ', ' ', ' ']
[' ', ' ', ' ', ' ', ' ']
[' ', ' ', ' ', ' ', ' ']
[' ', ' ', ' ', ' ', ' ']
[' ', ' ', ' ', ' ', ' ']

Human Turn


KeyboardInterrupt: Interrupted by user